<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#Lgbm" data-toc-modified-id="Lgbm-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Lgbm</a></span><ul class="toc-item"><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import re
from pymystem3 import Mystem
import time
import warnings
warnings.filterwarnings('ignore')
import string
import spacy
import sys
from tqdm.auto import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0,nrows=50000)

In [3]:
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 50054
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   toxic   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [5]:
df['toxic'].value_counts()

0    44853
1     5147
Name: toxic, dtype: int64

В столбце text присутсвуют слова с заглавной буквы, комментарии написанные капсом и посторонние знаки с отступами \n. отношение токсичных кооментариев к обычным 1/10.

In [6]:
df.describe()

,toxic
count,50000.000000
mean,0.102940
std,0.303884
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub("[^a-zA-Z]"," ", text)
    text = text.strip(' ')
    return text
df['text'] = df['text'].map(lambda x: clean_text(x))
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m s...,0
2,hey man i m really not trying to edit war it...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember...,0


In [8]:
nlp = spacy.load('en_core_web_sm')
def lemmatize(text):
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)   
    return lemm_text

In [9]:
def lemmatize(text):
    global nlp
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])
df['text'] = df['text'].progress_apply(lemmatize) 
df.head()

  0%|          | 0/50000 [00:00<?, ?it/s]

,text,toxic
0,explanation why the edit make under my usernam...,0
1,d aww he match this background colour I m se...,0
2,hey man I m really not try to edit war it ...,0


После очисти от лишних знаков и последующей лемматизации данные подготовлены для дальнейшего анализа.

## Обучение

Разделим выборки на обучающую и тестовую

In [10]:
feature = df.drop('toxic', axis=1)
target = df['toxic']
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size = 0.3, random_state=12345)

## LogisticRegression

In [11]:
lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345, max_iter = 10000))])
params = {'clf__C': [10, 20, 100],
          'clf__class_weight': ['balanced', None]}
lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1)
lr_grid.fit(feature_train['text'], target_train)
lr_best_paramms = lr_grid.best_params_
print(lr_best_paramms)
print(lr_grid.best_score_)

{'clf__C': 20, 'clf__class_weight': 'balanced'}
0.7545638174238231


## Lgbm 

In [13]:
lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=12345))])
params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.15,0.25],
  'clf__max_depth': [8, 10, 30]}
lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1)
lgb_grid.fit(feature_train['text'], target_train)
lgb_best_params = lgb_grid.best_params_
print(lgb_best_params)
print(lgb_grid.best_score_)

{'clf__learning_rate': 0.15, 'clf__max_depth': 30, 'clf__n_estimators': 200}
0.7462941411000178


### RandomForestClassifier

In [15]:
rfc_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', RandomForestClassifier(random_state=12345))])
params = {
  'clf__n_estimators': [20, 100],
  'clf__max_depth': [8, 10, None]}
rfc_grid = GridSearchCV(estimator=rfc_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1)
rfc_grid.fit(feature_train['text'], target_train)
rfc_best_params = rfc_grid.best_params_
print(rfc_best_params)
print(rfc_grid.best_score_)

{'clf__max_depth': None, 'clf__n_estimators': 100}
0.7031738658621839


Рассчитали метрики f1 для тренировочной выборок

LogisticRegression на тренировочной выборке f1 = 0.755

Lgbm на тренировочной выборке f1 = 0.746

RandomForestClassifier на тренировочной выборке f1 = 0.703

Наилучший результат у LogisticRegression расчитаем метрику f1 на тестовой выборке

In [17]:
lr_pred = lr_grid.best_estimator_.predict(feature_test['text'])
f1_score(target_test, lr_pred)

0.7557894736842106

## Выводы

В ходе работы была выполнена лематизация, чистка текста библиотекой spacy, так же очистили от мусоа и неинформативных слов и символов, оставим только слова. В связи с ограничениями платформы пришлось использовать 50000 тыс строк из начального датафрейма.    

Было обучено несколько алгоритмов, подобраны гипперпараметры посчитана f1  на тестовой выборке, получено значение 0,756. 

Интернет магазину викишоп будет предложен именно этот алгоритм для классификации коментариев.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны